# Code


In [ ]:
import torch
# Install required packages.
import os
os.environ['TORCH'] = torch.__version__

!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 71.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/nightly/torch-2.5.1+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 6.5 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-sjeu6oc3
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-sjeu6oc3
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit bb6601c8666e69205a7a4d1d981b771e9bae6880
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.7.0-py3-none-any.whl 

In [ ]:
!pip install mamba-ssm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ninja-1.11.1.3-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  

In [ ]:
!pip install triton

In [ ]:
import argparse
import os.path as osp
from typing import Any, Dict, Optional

import torch
from torch.nn import (
    BatchNorm1d,
    Embedding,
    Linear,
    ModuleList,
    LeakyReLU,
    Sequential,
    Sigmoid
)
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINConv, global_add_pool
import inspect
from typing import Any, Dict, Optional

import torch.nn.functional as F
from torch import Tensor
from torch.nn import Dropout, Linear, Sequential
import torch.nn as nn
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import reset
from torch_geometric.nn.resolver import (
    activation_resolver,
    normalization_resolver,
)
from torch_geometric.typing import Adj
from torch_geometric.utils import to_dense_batch
from torch_geometric.loader import DataLoader, LinkNeighborLoader
from mamba_ssm import Mamba, Mamba2
from torch_geometric.utils import degree, sort_edge_index

In [ ]:
def permute_within_batch(x, batch):
    # Enumerate over unique batch indices
    unique_batches = torch.unique(batch)

    # Initialize list to store permuted indices
    permuted_indices = []

    for batch_index in unique_batches:
        # Extract indices for the current batch
        indices_in_batch = (batch == batch_index).nonzero().squeeze()

        # Permute indices within the current batch
        permuted_indices_in_batch = indices_in_batch[torch.randperm(len(indices_in_batch))]

        # Append permuted indices to the list
        permuted_indices.append(permuted_indices_in_batch)

    # Concatenate permuted indices into a single tensor
    permuted_indices = torch.cat(permuted_indices)

    return permuted_indices

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def str_list_to_float(str_list):
    return [float(item) for item in str_list]


def str_list_to_int(str_list):
    return [int(item) for item in str_list]


def read_embeddings(filename):
    """read pretrained node embeddings
    """

    with open(filename, "r") as f:
        n_node, n_embed = str_list_to_int(f.readline().split())
        lines = f.readlines()[1:]  # skip the first line
        embedding_matrix = np.random.rand(n_node, n_embed)
        for line in lines:
            emd = line.split()
            embedding_matrix[int(emd[0]), :] = str_list_to_float(emd[1:])
        return torch.tensor(embedding_matrix, dtype=torch.float32)


def read_edges_from_file(filename):
    with open(filename, "r") as f:
        lines = f.readlines()

    if not lines:  # Check if file is empty
        raise ValueError(f"Edge file {filename} is empty!")

    edges = [list(map(int, line.split())) for line in lines]  # Convert to int list
    edges = torch.tensor(edges, dtype=torch.long)  # Ensure proper tensor type

    if edges.shape[1] != 2:  # Ensure it's a 2-column tensor
        raise ValueError(f"Edge file {filename} has incorrect shape: {edges.shape}")

    return edges


In [ ]:
train_edges_filename = "/content/drive/MyDrive/Emb_Data/Data/Biogrid-human/bio-grid-human_train.txt"
test_edges_filename = "/content/drive/MyDrive/Emb_Data/Data/Biogrid-human/bio-grid-human_test.txt"
emb_filename = "/content/drive/MyDrive/Emb_Data/Emb/biogrid-human/DeepWalk/fuzzy_emb.txt"
n_nodes = 5242
n_embed = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

In [ ]:
from torch_geometric.data import Data

train_edges = read_edges_from_file(train_edges_filename).to(device).contiguous().T
test_edges = read_edges_from_file(test_edges_filename).to(device).contiguous().T
train_edges_label = torch.ones(train_edges.size(1), device=device, dtype=torch.long)
test_edges_label = torch.ones(test_edges.size(1), device=device, dtype=torch.long)
node_embeddings = read_embeddings(emb_filename).to(device)

train_data = Data(x=node_embeddings, edge_index=train_edges.contiguous())
test_data = Data(x=node_embeddings, edge_index=test_edges.contiguous())

In [ ]:
print("train_edges shape:", train_edges.shape)
print("train_edges min:", train_edges.min().item(), "max:", train_edges.max().item())
print("train_edges dtype:", train_edges.dtype)


train_edges shape: torch.Size([2, 56127])
train_edges min: 0 max: 9435
train_edges dtype: torch.int64


In [ ]:
train_loader = LinkNeighborLoader(data = train_data,
                          num_neighbors = [20, 20],
                          batch_size=64,
                          neg_sampling='binary',
                          shuffle=True,
                          edge_label_index = train_data.edge_index.contiguous(),
                          edge_label=torch.ones(train_data.edge_index.size(1)))
test_loader = LinkNeighborLoader(data = test_data,
                          num_neighbors = [20, 20],
                          batch_size=64,
                          neg_sampling = 'binary',
                          shuffle=False,
                          edge_label_index = test_data.edge_index.contiguous(),
                          edge_label=torch.ones(test_data.edge_index.size(1)))

In [ ]:
class GPSConv(torch.nn.Module):

    def __init__(
        self,
        channels: int,
        conv: Optional[MessagePassing],
        heads: int = 1,
        dropout: float = 0.0,
        attn_dropout: float = 0.0,
        act: str = 'relu',
        att_type: str = 'transformer',
        order_by_degree: bool = False,
        shuffle_ind: int = 0,
        d_state: int = 16,
        d_conv: int = 4,
        act_kwargs: Optional[Dict[str, Any]] = None,
        norm: Optional[str] = 'batch_norm',
        norm_kwargs: Optional[Dict[str, Any]] = None,
    ):
        super().__init__()

        self.channels = channels
        self.conv = conv
        self.heads = heads
        self.dropout = dropout
        self.att_type = att_type
        self.shuffle_ind = shuffle_ind
        self.order_by_degree = order_by_degree

        assert (self.order_by_degree==True and self.shuffle_ind==0) or (self.order_by_degree==False), f'order_by_degree={self.order_by_degree} and shuffle_ind={self.shuffle_ind}'

        if self.att_type == 'transformer':
            self.attn = torch.nn.MultiheadAttention(
                channels,
                heads,
                dropout=attn_dropout,
                batch_first=True,
            )
        if self.att_type == 'mamba':
            self.self_attn = Mamba(
                d_model=channels,
                d_state=d_state,
                d_conv=d_conv,
                expand=1,
            )

        self.mlp = Sequential(
            Linear(channels, channels * 2),
            activation_resolver(act, **(act_kwargs or {})),
            Dropout(dropout),
            Linear(channels * 2, channels),
            Dropout(dropout),
        )

        norm_kwargs = norm_kwargs or {}
        self.norm1 = normalization_resolver(norm, channels, **norm_kwargs)
        self.norm2 = normalization_resolver(norm, channels, **norm_kwargs)
        self.norm3 = normalization_resolver(norm, channels, **norm_kwargs)

        self.norm_with_batch = False
        if self.norm1 is not None:
            signature = inspect.signature(self.norm1.forward)
            self.norm_with_batch = 'batch' in signature.parameters

    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        if self.conv is not None:
            self.conv.reset_parameters()
        self.attn._reset_parameters()
        reset(self.mlp)
        if self.norm1 is not None:
            self.norm1.reset_parameters()
        if self.norm2 is not None:
            self.norm2.reset_parameters()
        if self.norm3 is not None:
            self.norm3.reset_parameters()

    def forward(
        self,
        x: Tensor,
        edge_index: Adj,
        batch: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> Tensor:
        r"""Runs the forward pass of the module."""
        hs = []
        if self.conv is not None:  # Local MPNN.
            h = self.conv(x, edge_index, **kwargs)
            h = F.dropout(h, p=self.dropout, training=self.training)
            h = h + x
            if self.norm1 is not None:
                if self.norm_with_batch:
                    h = self.norm1(h, batch)
                else:
                    h = self.norm1(h)
            hs.append(h)

        ### Global attention transformer-style model.
        if self.att_type == 'transformer':
            h, mask = to_dense_batch(x, batch)
            h, _ = self.attn(h, h, h, key_padding_mask=~mask, need_weights=False)
            h = h[mask]

        if self.att_type == 'mamba':

            if self.order_by_degree:
                deg = degree(edge_index[0], x.shape[0]).to(torch.long)
                print(f"shape of deg : {deg.shape}")
                order_tensor = torch.stack([batch, deg], 1).T
                _, x = sort_edge_index(order_tensor)

            if self.shuffle_ind == 0:
                h, mask = to_dense_batch(x)
                h = self.self_attn(h)[mask]
            else:
                mamba_arr = []
                for _ in range(self.shuffle_ind):
                    h_ind_perm = permute_within_batch(x, batch)
                    h_i, mask = to_dense_batch(x[h_ind_perm], batch)
                    h_i = self.self_attn(h_i)[mask][h_ind_perm]
                    mamba_arr.append(h_i)
                h = sum(mamba_arr) / self.shuffle_ind
        ###

        h = F.dropout(h, p=self.dropout, training=self.training)
        h = h + x  # Residual connection.
        if self.norm2 is not None:
            if self.norm_with_batch:
                h = self.norm2(h, batch=batch)
            else:
                h = self.norm2(h)
        hs.append(h)

        out = sum(hs)  # Combine local and global outputs.

        out = out + self.mlp(out)
        if self.norm3 is not None:
            if self.norm_with_batch:
                out = self.norm3(out, batch=batch)
            else:
                out = self.norm3(out)

        return out

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.channels}, '
                f'conv={self.conv}, heads={self.heads})')

In [ ]:
class GraphModel(torch.nn.Module):
    def __init__(self, channels: int, pe_dim: int, num_layers: int, model_type: str, shuffle_ind: int, d_state: int, d_conv: int, order_by_degree: False):
        super().__init__()

        self.node_emb = Embedding(28, channels - pe_dim)
        self.pe_lin = Linear(20, pe_dim)
        self.pe_norm = BatchNorm1d(20)
        self.edge_emb = Embedding(4, channels)
        self.model_type = model_type #specifies which architecutre we are using
        self.shuffle_ind = shuffle_ind
        self.order_by_degree = order_by_degree
        self.dropout = torch.nn.Dropout(0.3)
        self.sigmoid = Sigmoid()
        self.convs = ModuleList()
        for _ in range(num_layers):
            nn = Sequential(
                Linear(channels, channels),
                LeakyReLU(0.01),
                Linear(channels, channels),
            )
            if self.model_type == 'gine':
                conv = GINConv(nn) #node features and edge indices needed

            if self.model_type == 'mamba':
                conv = GPSConv(channels, GINConv(nn), heads=4, attn_dropout=0.5,
                               att_type='mamba',
                               shuffle_ind=self.shuffle_ind,
                               order_by_degree=self.order_by_degree,
                               d_state=d_state, d_conv=d_conv)

            if self.model_type == 'transformer':
                conv = GPSConv(channels, GINConv(nn), heads=4, attn_dropout=0.5, att_type='transformer')

            # conv = GINEConv(nn)
            self.convs.append(conv)

        self.mlp = Sequential(
            self.dropout,
            Linear(channels, channels // 2),
            LeakyReLU(0.01),
            Linear(channels // 2, channels // 4),
            LeakyReLU(0.01),
            self.dropout,
            Linear(channels // 4, 1),
        )
        self.linear = Linear(n_embed, channels)
    def forward(self, x, edge_index, edge_label_index):
        num_nodes = x.size(0)
        #print(f"Max edge index: {edge_index.max()}, Num nodes: {num_nodes}")
        assert edge_index.max() < num_nodes, "Edge index out of bounds!"
        x = self.dropout(x)
        x = self.linear(x)
        for conv in self.convs:
            if self.model_type == 'gine':
                x = conv(x, edge_index)
            else:
                x = conv(x, edge_index)

        pred = self.mlp(x)
        # Step 2: Get embeddings for the source and target nodes of the edges
        edge_src = x[edge_label_index[0]]  # Embeddings of source nodes
        edge_dst = x[edge_label_index[1]]  # Embeddings of target nodes

        # Step 3: Compute edge scores (using dot product as the decoder)
        # Use the dot product as the decoder for link prediction
        edge_features = edge_src * edge_dst  # Element-wise multiplication
        pred = self.mlp(edge_features)       # Pass through MLP for final prediction
        pred = pred.squeeze(-1)              # Remove last dimension if it exists


        return pred

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = GraphModel(channels=64, pe_dim=4, num_layers=2,
                   model_type='mamba',
                   shuffle_ind=0, order_by_degree=False,
                   d_conv=2, d_state=4,
                  ).to(device)

for name, param in model.named_parameters():
    if torch.isnan(param).any() or torch.isinf(param).any():
        print(f"NaN or Inf detected in parameter: {name}")

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)
epochs = 10


def evaluate(loader, model):
    """Evaluate the model and calculate metrics."""
    model.eval()
    total_loss = total_examples = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for sampled_data in tqdm(loader, desc="Evaluating"):
            # Move data to device
            sampled_data = sampled_data.to(device)

            # Forward pass
            pred = model(sampled_data.x, sampled_data.edge_index, sampled_data.edge_label_index)
            # Ground truth labels
            ground_truth = sampled_data.edge_label

            # Apply binary cross entropy with logits
            loss = criterion(pred, ground_truth.float())
            total_loss += float(loss) * pred.numel()
            total_examples += pred.numel()

            # Collect predictions and labels for metrics
            all_preds.append(torch.sigmoid(pred).cpu().numpy())
            all_labels.append(ground_truth.cpu().numpy())

    # Concatenate all predictions and labels
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    # Convert probabilities to binary predictions using threshold 0.5
    binary_preds = (all_preds >= 0.5).astype(int)

    # Calculate metrics
    accuracy = accuracy_score(all_labels, binary_preds)
    f1 = f1_score(all_labels, binary_preds)
    precision = precision_score(all_labels, binary_preds)
    recall = recall_score(all_labels, binary_preds)

    avg_loss = total_loss / total_examples
    return avg_loss, accuracy, f1, precision, recall

# Training loop
for epoch in range(epochs):
    total_loss = total_examples = 0
    all_preds = []
    all_labels = []

    model.train()
    for sampled_data in tqdm(train_loader, desc=f"Epoch {epoch}"):
        optimizer.zero_grad()

        # Move `sampled_data` to the respective `device`
        # Move sampled_data to the appropriate device
        sampled_data = sampled_data.to(device)

        #print(f"Shape of input tensor : {sampled_data.x.shape}")
        # Run `forward` pass of the model
        pred = model(sampled_data.x, sampled_data.edge_index, sampled_data.edge_label_index)

        # Ground truth labels for link prediction (0 or 1)
        ground_truth = sampled_data.edge_label
        #print(f"pred shape: {pred.shape}, ground_truth shape: {ground_truth.shape}")

        # Apply binary cross entropy with logits
        loss = criterion(pred, ground_truth.float())

        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()

        # Collect predictions and labels for metrics
        all_preds.append(torch.sigmoid(pred).detach().cpu().numpy())  # Apply sigmoid to get probabilities
        all_labels.append(ground_truth.cpu().numpy())

    # Concatenate all predictions and labels for the epoch
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    # Convert probabilities to binary predictions using threshold 0.5
    binary_preds = (all_preds >= 0.5).astype(int)

    # Calculate training metrics
    train_loss = total_loss / total_examples
    train_accuracy = accuracy_score(all_labels, binary_preds)
    train_f1 = f1_score(all_labels, binary_preds)
    train_precision = precision_score(all_labels, binary_preds)
    train_recall = recall_score(all_labels, binary_preds)

    print(f"Epoch: {epoch:03d}, Loss: {train_loss:.4f}, "
          f"Accuracy: {train_accuracy:.4f}, F1: {train_f1:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")

    # Save training metrics to the file
    with open("/content/drive/MyDrive/GNN-Results/biogrid-human/DeepWalk/GraphMamba/training_metrics_fuzzy.txt", "a") as file:
        file.write(f"{epoch:03d}\t{train_loss:.4f}\t{train_accuracy:.4f}\t"
                   f"{train_f1:.4f}\t{train_precision:.4f}\t{train_recall:.4f}\n")

    # Evaluation at the end of each epoch
    eval_loss, eval_accuracy, eval_f1, eval_precision, eval_recall = evaluate(test_loader, model)
    print(f"Evaluation - Loss: {eval_loss:.4f}, Accuracy: {eval_accuracy:.4f}, "
          f"F1: {eval_f1:.4f}, Precision: {eval_precision:.4f}, Recall: {eval_recall:.4f}\n")

    # Save evaluation metrics to the file
    with open("/content/drive/MyDrive/GNN-Results/biogrid-human/DeepWalk/GraphMamba/evaluation_metrics_fuzzy.txt", "a") as file:
        file.write(f"Eval\t{eval_loss:.4f}\t{eval_accuracy:.4f}\t"
                   f"{eval_f1:.4f}\t{eval_precision:.4f}\t{eval_recall:.4f}\n")


cuda


Epoch 0: 100%|██████████| 877/877 [00:11<00:00, 73.15it/s]


Epoch: 000, Loss: 0.2207, Accuracy: 0.9137, F1: 0.9172, Precision: 0.8814, Recall: 0.9562


Evaluating: 100%|██████████| 98/98 [00:00<00:00, 200.21it/s]


Evaluation - Loss: 0.2133, Accuracy: 0.9323, F1: 0.9279, Precision: 0.9920, Recall: 0.8716



...

Evaluation - Loss: 0.0868, Accuracy: 0.9702, F1: 0.9698, Precision: 0.9827, Recall: 0.9572



Epoch 9: 100%|██████████| 877/877 [00:12<00:00, 72.27it/s]


Epoch: 009, Loss: 0.0639, Accuracy: 0.9793, F1: 0.9795, Precision: 0.9715, Recall: 0.9876


Evaluating: 100%|██████████| 98/98 [00:00<00:00, 188.68it/s]

Evaluation - Loss: 0.0718, Accuracy: 0.9769, F1: 0.9772, Precision: 0.9643, Recall: 0.9905

